In [ ]:
import notebooks.exploratory_variables as ev


# import pandas_settings
from src import PathOrganizer
from src import Model_Exploration_Tool as ES
from src import ExploratoryAid as EA

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import cobra
import re

# %matplotlib inline

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)

po = PathOrganizer()

##############################

draftModels = [
    "amuciniphila_draft_xml.xml",
    "amucosicola_draft_xml.xml",
    "bcaecimuris_draft_xml.xml",
    "bpseudococcoides_draft_xml.xml",
    "cinnocuum_draft_xml.xml",
    "cramosum_draft_xml.xml",
    "eclostridioformis_draft_xml.xml",
    "efaecalis_draft_xml.xml",
    "ecoli_draft_xml.xml",
    "emuris_draft_xml.xml",
    "fplautii_draft_xml.xml",
    "fbutyricus_draft_xml.xml",
    "lreuteri_draft_xml.xml",
    "lmurinus_draft_xml.xml",
    "mschaedleri_draft_xml.xml",
    "mintestinale_draft_xml.xml",
    "pgoldsteinii_draft_xml.xml",
    "tmuris_draft_xml.xml",
    "xrodentium_draft_xml.xml",
]

In [ ]:
ea = EA()

In [ ]:
crucial, variable = ea.generate_essentialMetabolites_dictionaries(
    targetModels=ev.draftModels,
    mediumMetabolites=ev.kwoji_medium,
    mediumMetabolitesDataframe=ev.kwoji_medium_df,
    closedMetabolites=ev.closed_uptake,
)

best_eMets = ea.essentialMetabolites_report(
    targetModels=ev.draftModels,
    mediumMetabolites=ev.kwoji_medium,
    mediumMetabolitesDataframe=ev.kwoji_medium_df,
    closedMetabolites=ev.closed_uptake,
    crucial_eMetabolites=crucial,
    variable_eMetabolites=variable,
)

In [ ]:
results_MA = (
    ea.metabolite_assay_testing(
        targetModels=ev.draftModels,
        mediumMetabolites=ev.kwoji_medium,
        closedMetabolites=ev.closed_uptake,
        targetModels_EssentialMetabolites=best_eMets,
        assayMetabolitesDataframe=ev.biolog_met_df,
        mediumMetabolitesDataframe=ev.kwoji_medium_df,
        optimizationType="FBA",
    )
)

In [ ]:
okay = results_MA.columns.get_level_values(level=0).unique()

In [ ]:
list(okay)

for ok in list(okay):
    
    print(ok)
    # display(results_MA[ok])
    display(results_MA[ok].sort_values(by=[("ATPm", "FBA")], ascending=False,))

In [ ]:
ATPm_target = results_MA.loc[:,([f for f in results_MA.columns.get_level_values(level=0).unique()],"ATPm")]

In [ ]:
ATPm_analysis_DF = ATPm_target.droplevel(axis=1, level=1)

In [ ]:
ATPm_analysis_FVA = ATPm_analysis_DF.loc[:, ([f for f in results_MA.columns.get_level_values(level=0).unique()],["FVA Min", "FVA Max"])]
ATPm_analysis_FBA = ATPm_analysis_DF.loc[:, ([f for f in results_MA.columns.get_level_values(level=0).unique()],["FBA"])].droplevel(axis=1,level=1)

In [ ]:
ATPm_analysis_FBA

In [ ]:
for strain in ATPm_analysis_FBA.columns.unique():
    
    water = ATPm_analysis_FBA.loc["Water", strain]
        
    ATPm_analysis_FBA[strain] = np.select(condlist=[ATPm_analysis_FBA[strain] <= water * 1.02, ATPm_analysis_FBA[strain] <= water * 2], choicelist=[0, 1], default=2)
    
display(ATPm_analysis_FBA)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns
from notebooks import biolog_variables as bv

In [ ]:
okabe_ito_rgb = [
    colors.to_rgb("#DDDDDD"),  # light grey
    colors.to_rgb("#7E2954"),  # sky blue
    colors.to_rgb("#009E73"),  # bluish green
]

cmap = colors.ListedColormap(okabe_ito_rgb)
norm = colors.BoundaryNorm(boundaries=[-0.5, 0.5, 1.5, 2.5], ncolors=3)

strain_IDs = ATPm_analysis_FBA.columns.get_level_values(level=0).unique().tolist()
strain_names = bv.strain_ids.loc[bv.strain_ids["DSMZ-number"].isin(strain_IDs), "Strain"].tolist()

In [ ]:
met_groups = ev.biolog_met_complete_df.loc[:, ("Metabolite", "Group")]
met_groups = met_groups.set_index("Metabolite")

group_list = met_groups["Group"].to_list()

Group_colors = {
        "Miscellaneous / Other": "#000000",
        "Carbohydrates & Derivatives": "#D55E00",
        "Organic Acids & Derivatives": "#56B4E9",
        "Amino Acids & Peptides": "#CC79A7",
        "Nucleosides & Nucleotides": "#0072B2",
    }

metGroup_colors = [Group_colors[x] for x in group_list]

In [ ]:
sns.set_theme(rc={"figure.dpi": 600})
sns.set_theme(rc={"xtick.bottom": True, "ytick.left": True})
sns.set_theme(font_scale=0.75)
g = sns.clustermap(
    data=ATPm_analysis_FBA.T,
    figsize=(17, 6),
    annot=False,
    dendrogram_ratio=(0.05, 0.2),
    col_cluster=False,
    yticklabels=strain_names,
    row_cluster=False,
    cmap=cmap,
    norm=norm,
    linewidth=0.75,
    col_colors=metGroup_colors,
)

g.ax_cbar.remove()
g.ax_heatmap.yaxis.set_ticks_position("left")
g.ax_heatmap.yaxis.set_label_position("left")

col_pos = g.ax_col_colors.get_position()
heatmap_pos = g.ax_heatmap.get_position()

# Shift col_colors axis below heatmap
g.ax_col_colors.set_position(
    [
        heatmap_pos.x0,
        heatmap_pos.y0 - 0.01,  # small offset
        heatmap_pos.width,
        col_pos.height,
    ]
)

# Adjust heatmap to shift it up if necessary
g.ax_heatmap.set_position(
    [
        heatmap_pos.x0,
        heatmap_pos.y0 + 0.01,
        heatmap_pos.width,
        heatmap_pos.height,
    ]
)
# g.ax_heatmap.xaxis.set_tick_params(which="minor")

g.ax_heatmap.set_xlabel(
    "Metabolites",
    fontdict={
        "size": 12,
        "weight": "bold",
    },
)
g.ax_heatmap.set_ylabel(
    "Strains",
    fontdict={
        "size": 12,
        "weight": "bold",
    },
)

plt.show()